CoordinateSystem: PROJCS["TUREF / TM27",GEOGCS["TUREF",DATUM["Turkish National Reference Frame",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","1057"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9102"]],AUTHORITY["EPSG","5252"]],PROJECTION["Transverse_Mercator",AUTHORITY["EPSG","9807"]],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",27],PARAMETER["scale_factor",1],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AUTHORITY["EPSG","5253"]]


Label,X/Easting,Y/Northing,Z/Altitude,Yaw,Pitch,Roll,X_est,Y_est,Z_est,Yaw_est,Pitch_est,Roll_est
DJI_0170.JPG,502690.655770,4264468.328425,129.600041,359.200000,0.100000,-0.000000,502690.450010,4264466.363887,80.363514,359.780665,0.267668,-0.937320
DJI_0171.JPG,502690.269529,4264488.756772,129.300041,357.900000,0.100000,-0.000000,502690.040010,4264487.472347,80.303678,358.844635,0.170521,-0.917314

In [18]:
import numpy as np
import cv2 

# ---------------------------
# 1. Kamera Parametreleri
# ---------------------------

# Kamera iç parametreleri:
focal_length_mm = 9.46        # mm cinsinden
pixel_size_mm   = 0.00241228  # mm/pixel
image_width     = 5472        # piksel
image_height    = 3648        # piksel

# Principal point (görüntü merkezi)
cx_pixel = image_width / 2.0
cy_pixel = image_height / 2.0

# Odak uzaklığının piksel cinsine çevrilmesi:
focal_length_px = focal_length_mm / pixel_size_mm

# Kamera iç matrisi (K) - piksel cinsinden:
K = np.array([[focal_length_px, 0,               cx_pixel],
              [0,               focal_length_px, cy_pixel],
              [0,               0,               1]], dtype=float)


# Kamera 1 (sol fotoğraf):
C1 = np.array([502690.450010, 4264466.363887, 80.363514], dtype=float)
omega1, phi1, kappa1 = np.deg2rad(-0.937320), np.deg2rad(0.267668), np.deg2rad(359.780665)

# Kamera 2 (sağ fotoğraf):
C2 = np.array([502690.040010, 4264487.472347, 80.303678], dtype=float)
omega2, phi2, kappa2 = np.deg2rad(-0.917314), np.deg2rad(0.170521), np.deg2rad(358.844635)

print("Kamera iç matrisi (K):\n", K)
print("\nKamera 1 merkezi (C1):", C1)
print("Kamera 2 merkezi (C2):", C2)


Kamera iç matrisi (K):
 [[3.92160114e+03 0.00000000e+00 2.73600000e+03]
 [0.00000000e+00 3.92160114e+03 1.82400000e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Kamera 1 merkezi (C1): [5.02690450e+05 4.26446636e+06 8.03635140e+01]
Kamera 2 merkezi (C2): [5.02690040e+05 4.26448747e+06 8.03036780e+01]


In [19]:
def rotation_matrix_from_omega_phi_kappa(omega, phi, kappa):
    """
    Euler açıları (radyan cinsinden: omega, phi, kappa) kullanılarak 
    X, Y, Z eksenleri etrafında sırasıyla dönüş matrisini oluşturur.
    Dönüş sırası: önce X, sonra Y, son olarak Z ekseni etrafında.
    """
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(omega), -np.sin(omega)],
                   [0, np.sin(omega),  np.cos(omega)]])
    Ry = np.array([[ np.cos(phi), 0, np.sin(phi)],
                   [0,           1, 0],
                   [-np.sin(phi), 0, np.cos(phi)]])
    Rz = np.array([[np.cos(kappa), -np.sin(kappa), 0],
                   [np.sin(kappa),  np.cos(kappa), 0],
                   [0,              0,             1]])
    # Toplam dönüş matrisi: R = Rz * Ry * Rx
    return Rz @ Ry @ Rx

# Kameralar için dönüş matrisleri:
R1 = rotation_matrix_from_omega_phi_kappa(omega1, phi1, kappa1)
R2 = rotation_matrix_from_omega_phi_kappa(omega2, phi2, kappa2)

def pixel_to_ray(u, v, K, pixel_size_mm, focal_length_mm):
    """
    Verilen piksel koordinatları (u, v) için:
    - Piksel ofsetini mm cinsine çevir,
    - Kamera düzlemindeki nokta: [x, y, focal_length_mm] (z ekseni optik eksendir),
    - Vektör normalize edilerek birim vektör elde edilir.
    """
    cx = K[0, 2]
    cy = K[1, 2]
    x_mm = (u - cx) * pixel_size_mm
    y_mm = (v - cy) * pixel_size_mm
    ray_cam = np.array([x_mm, y_mm, focal_length_mm], dtype=float)
    return ray_cam / np.linalg.norm(ray_cam)

print("Kamera 1 dönüş matrisi (R1):\n", R1)
print("\nKamera 2 dönüş matrisi (R2):\n", R2)

Kamera 1 dönüş matrisi (R1):
 [[ 0.99998176  0.00375117  0.00473363]
 [-0.00382807  0.99985916  0.01634059]
 [-0.00467167 -0.01635841  0.99985528]]

Kamera 2 dönüş matrisi (R2):
 [[ 0.99979227  0.02011334  0.00329797]
 [-0.02016347  0.99966952  0.01594621]
 [-0.00297615 -0.01600939  0.99986741]]


In [24]:
# Nesnenin her iki görüntüdeki piksel koordinatları
# Sol fotoğraf için:
u1, v1 = 3579, 1603
# Sağ fotoğraf için:
u2, v2 = 3584, 2723

# Her kamera için kamera koordinat sistemindeki ışın vektörleri (birim vektörler):
ray1_cam = pixel_to_ray(u1, v1, K, pixel_size_mm, focal_length_mm)
ray2_cam = pixel_to_ray(u2, v2, K, pixel_size_mm, focal_length_mm)

# Dünya koordinat sistemine geçiş:
# (Kamera koordinatından dünya koordinatına geçiş: ray_world = R^T * ray_cam)
d1 = R1.T @ ray1_cam
d2 = R2.T @ ray2_cam
d1 = d1 / np.linalg.norm(d1)
d2 = d2 / np.linalg.norm(d2)

print("Kamera 1'deki ışın vektörü (kamera koordinat sisteminde):\n", ray1_cam)
print("Kamera 2'deki ışın vektörü (kamera koordinat sisteminde):\n", ray2_cam)
print("\nKamera 1'deki ışın vektörü (dünya koordinat sisteminde):\n", d1)
print("Kamera 2'deki ışın vektörü (dünya koordinat sisteminde):\n", d2)


Kamera 1'deki ışın vektörü (kamera koordinat sisteminde):
 [ 0.20984408 -0.05501251  0.97618599]
Kamera 2'deki ışın vektörü (kamera koordinat sisteminde):
 [0.20623961 0.21864317 0.95376118]

Kamera 1'deki ışın vektörü (dünya koordinat sisteminde):
 [ 0.20549043 -0.07018645  0.9761391 ]
Kamera 2'deki ışın vektörü (dünya koordinat sisteminde):
 [0.19894963 0.20744994 0.95780142]


In [23]:
def triangulate_point_derenyi(C1, d1, C2, d2):
    """
    İki kamera merkezinden çıkan ve dünya koordinat sisteminde verilen
    iki birim yön vektörünün (d1, d2) en yakın noktalarını (skew lines) 
    hesaplar. 
    Derenyi'nin yönteminde, iki ışının en yakın yaklaşım noktalarının 
    ortalaması, nesnenin 3B konumunu verir.
    """
    # İki kamera merkezi arasındaki vektör:
    D = C2 - C1
    dot_d1_d2 = np.dot(d1, d2)
    denom = 1 - dot_d1_d2**2
    if np.abs(denom) < 1e-6:
        print("Işınlar neredeyse paralel, güvenilir sonuç alınamıyor.")
        return None
    mu  = ((np.dot(d1, D) * dot_d1_d2) - np.dot(d2, D)) / denom
    lambda_val = np.dot(d1, D) + dot_d1_d2 * mu

    P1 = C1 + lambda_val * d1
    P2 = C2 + mu * d2
    return (P1 + P2) / 2.0

# Üçgenleme sonucu nesnenin dünya koordinatları:
point_world = triangulate_point_derenyi(C1, d1, C2, d2)
if point_world is not None:
    X, Y, Z = point_world
    print("TUREF-TM/27 Sistemindeki Koordinatları: ")
    print("X = {:.2f}".format(X),"m")
    print("Y = {:.2f}".format(Y),"m")
    print("Z = {:.2f}".format(Z),"m")
else:
    print("Üçgenleme yapılamadı.")

TUREF-TM/27 Sistemindeki Koordinatları: 
X = 502674.94 m
Y = 4264471.63 m
Z = 7.13 m
